In [11]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from termcolor import colored
from PIL import Image
import dlib
#import cv2
from skimage.transform import resize
detector = dlib.get_frontal_face_detector()
#face_cascade = cv2.CascadeClassifier('/home/mckc/Downloads/opencv-2.4.13/data/haarcascades_GPU/haarcascade_frontalface_default.xml')

In [13]:
train = pd.read_csv('/home/mckc/image class//train.csv')
test = pd.read_csv('/home/mckc/image class//test.csv')

print 'the training data shape is ',train.shape
print 'the test data shape is ', test.shape
train_records = train.shape[0]
test_records = test.shape[0]
    
X_tr = np.zeros((1,3,224,224),dtype=np.uint8)
Y_tr =[]
iteration = 0
for i in train.values[:,0]:
    image = np.array(Image.open(i),dtype=np.uint8)
    faces = detector(image, 1)
#    faces = face_cascade.detectMultiScale(
#        cv2.cvtColor(image, cv2.COLOR_BGR2GRAY),
#        scaleFactor=1.1,
#        minNeighbors=5,
#        minSize=(30, 30),
#       flags=cv2.cv.CV_HAAR_SCALE_IMAGE
#    )
    if len(faces)==1:    
        for a,b in enumerate(faces):
#        for (x, y, w, h) in faces:
#            fac = resize(np.array(image)[y:(y+h),x:(x+h),:] ,(3,224,224)).reshape(-1,3,224,224)
            fac = resize(np.array(image)[b.top():b.bottom(),b.left():b.right(),:] ,(3,224,224)).reshape(-1,3,224,224)
        X_tr =  np.vstack((X_tr,fac))
        Y_tr = np.append(Y_tr,train.values[iteration,1])
    iteration+=1
    if iteration % 50==0:
        print colored((float(iteration)/train_records*100 ,' Percentage complete'), 'green')
    

X_tr = X_tr[1:,:,:]
                
iteration = 0      
X_ts = np.zeros((1,3,224,224),dtype=np.uint8)
Y_ts = []
for i in test.values[:,0]:
    image = np.array(Image.open(i),dtype=np.uint8)
    faces = detector(image, 1)
#    faces = face_cascade.detectMultiScale(
#        cv2.cvtColor(image, cv2.COLOR_BGR2GRAY),
#        scaleFactor=1.1,
#        minNeighbors=5,
#        minSize=(30, 30),
#       flags=cv2.cv.CV_HAAR_SCALE_IMAGE
#    )
    if len(faces)==1:    
        for a,b in enumerate(faces):
#        for (x, y, w, h) in faces:
#            fac = resize(np.array(image)[y:(y+h),x:(x+h),:] ,(3,224,224)).reshape(-1,3,224,224)
            fac = resize(np.array(image)[b.top():b.bottom(),b.left():b.right(),:] ,(3,224,224)).reshape(-1,3,224,224)
        X_ts =  np.vstack((X_ts,fac))
        Y_ts = np.append(Y_ts,test.values[iteration,1])
    iteration+=1
    if iteration % 50==0:
        print colored((float(iteration)/test_records*100 ,' Percentage complete'), 'green')
    
X_ts = X_ts[1:,:,:]
print 'the training file shape',X_tr.shape,Y_tr.shape
print 'the testing file shape',X_ts.shape,Y_ts.shape

the training data shape is  (114, 2)
the test data shape is  (38, 2)
(43.859649122807014, ' Percentage complete')
(87.71929824561403, ' Percentage complete')
the training file shape (29, 3, 224, 224) (29,)
the testing file shape (8, 3, 224, 224) (8,)


In [14]:
#from keras.datasets import cifar10
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
#(X_train, y_train), (X_test, y_test) = cifar10.load_data()
map, Y_number = np.unique(Y_tr, return_inverse=True)
Y_test_number = np.unique(Y_ts, return_inverse=True)[1]

Y_train = np_utils.to_categorical(Y_number, 2)
Y_test = np_utils.to_categorical(Y_test_number, 2)

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    )

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_tr.astype(np.float32))

In [15]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD

def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

model = VGG_16('/home/mckc/Downloads/vgg16_weights.h5')

model.layers.pop()
model.add(Dense(2, activation='softmax'))

sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
model.fit(X_tr.astype(np.float16),Y_train,verbose=1,validation_data=(X_ts.astype(np.float16),Y_test),nb_epoch=1000,batch_size=2)

Train on 29 samples, validate on 8 samples
Epoch 1/1000
29/29 [==============================] - 6s - loss: 0.6934 - acc: 0.5172 - val_loss: 0.6935 - val_acc: 0.3750
Epoch 2/1000
29/29 [==============================] - 6s - loss: 0.6928 - acc: 0.4828 - val_loss: 0.6934 - val_acc: 0.3750
Epoch 3/1000
29/29 [==============================] - 6s - loss: 0.6928 - acc: 0.5517 - val_loss: 0.6935 - val_acc: 0.3750
Epoch 4/1000
29/29 [==============================] - 6s - loss: 0.6938 - acc: 0.4138 - val_loss: 0.6935 - val_acc: 0.3750
Epoch 5/1000
29/29 [==============================] - 6s - loss: 0.6930 - acc: 0.4828 - val_loss: 0.6937 - val_acc: 0.3750
Epoch 6/1000
29/29 [==============================] - 6s - loss: 0.6935 - acc: 0.4828 - val_loss: 0.6938 - val_acc: 0.3750
Epoch 7/1000
29/29 [==============================] - 6s - loss: 0.6935 - acc: 0.4828 - val_loss: 0.6937 - val_acc: 0.3750
Epoch 8/1000
29/29 [==============================] - 6s - loss: 0.6932 - acc: 0.4828 - val_loss

KeyboardInterrupt: 

In [ ]:
# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(X_tr, Y_train, batch_size=10),validation_data=(X_ts.astype(np.float16),Y_test)
                    ,nb_worker=7,samples_per_epoch=len(X_tr), nb_epoch=1000, pickle_safe=True)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2, numpy as np

def VGG_19(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model


model = VGG_19('/home/mckc/Downloads/vgg19_weights.h5')

model.layers.pop()
model.add(Dense(2, activation='softmax'))

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

In [ ]:
model.fit(X_tr.astype(np.float16),Y_train,verbose=1,validation_data=(X_ts.astype(np.float16),Y_test)
          ,nb_epoch=1000,batch_size=2)

In [ ]:
# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(X_tr, Y_train, batch_size=10),validation_data=(X_ts.astype(np.float16),Y_test)
                    ,nb_worker=7,samples_per_epoch=len(X_tr), nb_epoch=1000, pickle_safe=True)